### Part 1

In [1]:
# (b)
from pymongo import MongoClient
# connect to the MongoDB
client = MongoClient('localhost', 27017)
db = client["msba"]
ip_addresses_collection = db["ip_addresses"]

# Create the document 
document = {"ip": "192.168.1.1", "city": "Davis", "zip": "95616"}

# Insert the document into the collection
ip_addresses_collection.insert_one(document)

InsertOneResult(ObjectId('65f686d76bae0b9bbc74271a'), acknowledged=True)

In [2]:
#(e)&(f)
import requests
access_key = '374b382be1e172404540afcdff024a8d'

# The list of IP addresses
ip_addresses = ['8.8.8.8', '128.120.0.25', '128.32.12.14', '64.165.72.144', '73.241.106.60']

# The base URL
base_url = 'http://api.ipstack.com/'

# make the API call
for ip in ip_addresses:
    url = f'{base_url}{ip}?access_key={access_key}&format=1'
    # Make the API call and store the response
    response = requests.get(url)
    print(response)
    # if it shows response [200], that means I successfully request the link
    # Convert JSON string
    data = response.json()
    city = data.get('city', 'Not found')
    zip_code = data.get('zip', 'Not found')
    print(f'IP Address: {ip}, City: {city}, ZIP: {zip_code}')


<Response [200]>
IP Address: 8.8.8.8, City: Glenmont, ZIP: 44628
<Response [200]>
IP Address: 128.120.0.25, City: Davis, ZIP: 95616
<Response [200]>
IP Address: 128.32.12.14, City: Berkeley, ZIP: 94705
<Response [200]>
IP Address: 64.165.72.144, City: Florin, ZIP: 95819
<Response [200]>
IP Address: 73.241.106.60, City: San Francisco, ZIP: 94109


In [3]:
# (g)
for ip in ip_addresses:
    #create url
    url = f'{base_url}{ip}?access_key={access_key}&format=1'
    # Make the API call and store the response
    response = requests.get(url)
    print(response)
    # if it shows response [200], that means I successfully request the link
    # Convert JSON string
    data = response.json()
    #prepare document to insert in mongodb
    document = {
        'ip': data.get('ip', ''),
        'city': data.get('city', ''),
        'zip': data.get('zip', '')
        }
    print(document)
    # Insert the document into the "ip_addresses" collection
    ip_addresses_collection.insert_one(document)

<Response [200]>
{'ip': '8.8.8.8', 'city': 'Glenmont', 'zip': '44628'}
<Response [200]>
{'ip': '128.120.0.25', 'city': 'Davis', 'zip': '95616'}
<Response [200]>
{'ip': '128.32.12.14', 'city': 'Berkeley', 'zip': '94705'}
<Response [200]>
{'ip': '64.165.72.144', 'city': 'Florin', 'zip': '95819'}
<Response [200]>
{'ip': '73.241.106.60', 'city': 'San Francisco', 'zip': '94109'}


### Part 2

In [40]:
###(a)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('https://www.ebay.com/')
time.sleep (3)


In [41]:
#(b)
#locate the search box and send input cell phone
input = driver.find_element(By.CSS_SELECTOR, 'input[id=gh-ac]')
input.send_keys('Cell Phones')
time.sleep (3)
#locate the search button and send request to click it
search = driver.find_element(By.CSS_SELECTOR, 'input[id=gh-btn]')
search.click()
time.sleep (3)

In [42]:
#(c)
#select unlocked filter
unlock = driver.find_element(By.XPATH, "//span[contains(text(),'Unlocked')]")
unlock.click()
time.sleep (3)
#select LG
Brand = driver.find_element(By.XPATH, "//span[contains(text(),'LG')]")
Brand.click()
time.sleep (3)
#screen size
screen = driver.find_element(By.XPATH, "//span[contains(text(),'6 in or More')]")
screen.click()
time.sleep (3)
#storage
storage = driver.find_element(By.XPATH, "//span[contains(text(),'128 GB')]")
storage.click()
time.sleep (3)
#lock
lock = driver.find_element(By.XPATH, "//span[contains(text(),'Factory Unlocked')]")
lock.click()
time.sleep (3)
#condition
condition = driver.find_element(By.XPATH, "//span[contains(text(),'Used')]")
condition.click()

In [14]:
#(d) save file
def saveString(html, filename):
	try:
		file = open(filename, "w", encoding='utf-8')
		file.write(str(html))
		file.close()
	except Exception as ex:
		print('Error: ' + str(ex))

def loadString(f="test.html"):
	try:
		html = open(f, "r", encoding='utf-8').read()
		return(html)
	except Exception as ex:
		print('Error: ' + str(ex))

In [44]:
from bs4 import BeautifulSoup
import requests
import time 

url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw=cell+phone&_sacat=0&Network=Unlocked&Brand=LG&Screen%2520Size=6%2520in%2520or%2520More&Storage%2520Capacity=128%2520GB&Lock%2520Status=Factory%2520Unlocked&_dcat=9355&rt=nc&LH_ItemCondition=3000'
headers = {'user-agent':'Mozilla/5.0'}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, 'html.parser')
result = soup.select('div', class_='s-item__info clearfix')
#using for loop to locate and find all links of products
links = []
for item in result:
    a_tag = item.select_one('a.s-item__link[href]')
    if a_tag:
        link = a_tag['href']
        #print(link)
        if link not in links:
            links.append(link)
links=links[1:] 

In [31]:
import re
import os
import time
#create a new folder to store the html
folder = "unlocked-lg-128gb-used.html"
if not os.path.exists(folder):
    os.makedirs(folder)
#store the html for products and name them using product id in ip address
for url in links:
    response = requests.get(url, headers=headers)
    time.sleep(5) 
    #use RegEx to locate the product id
    pattern = r'.*/([0-9]+)?.*'
    match = re.search(pattern, url)
    listing_id = match.group(1)
    file_path = os.path.join(folder, f"{listing_id}.html")
    saveString(response.text, file_path)
    print(f"Successfully saved {file_path}")

Successfully saved unlocked-lg-128gb-used.html/176115637080.html
Successfully saved unlocked-lg-128gb-used.html/204644040290.html
Successfully saved unlocked-lg-128gb-used.html/285739984400.html
Successfully saved unlocked-lg-128gb-used.html/166631586516.html
Successfully saved unlocked-lg-128gb-used.html/156011210029.html
Successfully saved unlocked-lg-128gb-used.html/386844549146.html
Successfully saved unlocked-lg-128gb-used.html/276372097860.html
Successfully saved unlocked-lg-128gb-used.html/404848070636.html
Successfully saved unlocked-lg-128gb-used.html/355548351308.html
Successfully saved unlocked-lg-128gb-used.html/225877078707.html
Successfully saved unlocked-lg-128gb-used.html/145546537282.html
Successfully saved unlocked-lg-128gb-used.html/196218696131.html
Successfully saved unlocked-lg-128gb-used.html/195533576859.html
Successfully saved unlocked-lg-128gb-used.html/285739984445.html
Successfully saved unlocked-lg-128gb-used.html/133865480682.html
Successfully saved unlock

In [39]:
from bs4 import BeautifulSoup
import requests
import time

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    html = loadString(file_path)
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup)
    #extract title for each store
    title = soup.find('title').text
    #extract seller
    seller = soup.select('span.ux-textspans.ux-textspans--BOLD')[0].text
    #extract rating
    rate = soup.select('div.d-stores-info-categories__container__info__section__item')[0].text
    #extract price
    price = soup.select('div.x-price-primary')[0].text
    #shipping info
    shipping = soup.select('div.ux-labels-values__values-content')[0].text
    if len(shipping) > 0:
        shipping = shipping 
    else: 
        shipping = 'No shipping info'
    #return info
    returninfo = soup.select('div.ux-labels-values__values-content')[3].text
    
    if len(returninfo) > 6:
        returninfo = returninfo 
    else: 
        returninfo = 'No return info' 
    #number of bids
    bid = soup.select('div.x-bid-count')
    if len(bid) > 0:
        bid = bid[0].text 
    else: 
        bid = 'No bid info'
 
    print(f'title:{title}')
    print(f'seller:{seller}')
    print(f'rating:{rate}')
    print(f'price:{price}')
    print(f'shipping:{shipping}')
    print(f'return:{returninfo}')
    print(f'no.bids:{bid}\n')
    


title:LG VELVET G9 Original Unlocked 6.8'' NFC G900N G900VM 6GB+128GB 16MP SmartPhone  | eBay
seller:|
rating:96.1% Positive Feedback
price:US $198.00
shipping:Free SpeedPAK Standard. See detailsfor shipping
return:14 days returns. Buyer pays for return shipping. See details- for more information about returns
no.bids:No bid info

title:LG Velvet 5G VERIZON AT&T - 128GB Smartphone Aurora Gray / Silver  *Excellent*  | eBay
seller:|
rating:98.5% Positive Feedback
price:US $139.00/ea
shipping:Free Expedited Shipping. See detailsfor shippingLocated in: Kensington, Maryland, United States
return:No return info
no.bids:No bid info

title:USED - LG V30+ H930DS 128GB (FACTORY UNLOCKED) 6.0" 4GB RAM Raspberry Rose  | eBay
seller:|
rating:99.5% Positive Feedback
price:US $129.99
shipping:Free Standard Shipping. See detailsfor shippingLocated in: Houston, Texas, United States
return:No return info
no.bids:No bid info

title:LG G8X ThinQ LM-G850UM / LMG850EMW Android 4G LTE 128GB 6GB RAM 6.4" Cell

title:LG Velvet 5G G900UM 128GB Aurora Silver 6.8" Screen (AT&T) -*Good**LGVelvet*LG98  | eBay
seller:|
rating:99.6% Positive Feedback
price:US $92.75
shipping:US $6.15 Standard Shipping. See detailsfor shippingLocated in: Winchester, Virginia, United States
return:No return info
no.bids:No bid info

title:LG V50 THINQ 5G V450PM/V500N 6gb 128gb Single Sim 6.4" Fingerprint Android 11  | eBay
seller:|
rating:94.4% Positive Feedback
price:US $195.00
shipping:US $22.00 Expedited Shipping. See detailsfor shipping
return:30 days returns. Buyer pays for return shipping. See details- for more information about returns
no.bids:No bid info

title:LG Velvet 5G LM-G900 128GB, AT&T, Aurora Silver *Grade C*  | eBay
seller:|
rating:98.5% Positive Feedback
price:US $155.95
shipping:Free Standard Shipping. See detailsfor shippingLocated in: Maplewood, New Jersey, United States
return:No return info
no.bids:No bid info

title:LG G8X ThinQ LM-G850 AT&T Only 128GB Black C Light Burn  | eBay
seller:|
ratin

title:Android LG G8X ThinQ LM-G850UM / LMG850EMW 6.4" 128GB 6GB RAM 4G LTE Phone  | eBay
seller:|
rating:90% Positive Feedback
price:US $167.01
shipping:Free Economy Shipping from Greater China to worldwide. See detailsfor shipping
return:Seller does not accept returns. See details- for more information about returns
no.bids:No bid info

title:LG G8X ThinQ LM-G850UM / LMG850EMW Android 6.4" 128GB 6GB RAM 4G LTE Phone  | eBay
seller:|
rating:84% Positive Feedback
price:US $167.82
shipping:Free Economy Shipping from Greater China to worldwide. See detailsfor shipping
return:Seller does not accept returns. See details- for more information about returns
no.bids:No bid info

title:LG V50 ThinnQ 5G LM-V500V 6GB RAM (Unlocked) Smartphone - Black read desc. 730309276089 | eBay
seller:|
rating:100% Positive Feedback
price:US $1,000.00
shipping:Free local pickup from Hemet, California, United States. See detailsfor pickup
return:Seller does not accept returns. See details- for more information 